# Importação dos dados

In [2]:
import pandas as pd
#Carregando a matriz de dados de treinamento
x_train= pd.read_csv('../data/x_train.csv')

In [3]:
y_train= pd.read_csv('../data/y_train.csv')

In [4]:
x_test= pd.read_csv('../data/x_test.csv')

In [5]:
y_test= pd.read_csv('../data/y_test.csv')

# 1. Treinamento do modelo e teste

## 1.1. Treinamento

In [7]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Dicionário de modelos e hiperparâmetros
modelos_parametros = {
    'Random Forest': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'n_estimators': [100, 200],
            'max_depth': [None, 10, 20]
        }
    },
    'SVM': {
        'modelo': SVC(),
        'parametros': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    },
    'KNN': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'n_neighbors': [3, 5, 7],
            'metric': ['euclidean', 'manhattan']
        }
    },
    'XGBoost': {
        'modelo': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        'parametros': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 6]
        }
    },
    'Decision Tree': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'max_depth': [None, 5, 10],
            'criterion': ['gini', 'entropy']
        }
    },
    'Logistic Regression': {
        'modelo': LogisticRegression(max_iter=1000),
        'parametros': {
            'C': [0.1, 1, 10],
            'penalty': ['l2']
        }
    },
    'Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 6]
        }
    }
}

# DataFrame para armazenar os resultados
resultados = []

# Loop de treinamento e avaliação
for nome, config in modelos_parametros.items():
    print(f"\n🔧 Treinando: {nome}")
    grid = GridSearchCV(config['modelo'], config['parametros'], cv=10, scoring='accuracy', n_jobs=-1)
    grid.fit(x_train, y_train)

    # Previsões com validação cruzada (k=10)
    y_pred = cross_val_predict(grid.best_estimator_, x_train, y_train, cv=10)

    # Métricas
    acc = accuracy_score(y_train, y_pred)
    prec = precision_score(y_train, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_train, y_pred, average='weighted')
    f1 = f1_score(y_train, y_pred, average='weighted')

    resultados.append({
        'Modelo': nome,
        'Acurácia': acc,
        'Precisão': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Melhores Hiperparâmetros': grid.best_params_
    })

# Criar DataFrame de resultados
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by='F1-Score', ascending=False)

# Exibir a tabela
print("\n📊 Resultados dos Modelos (validação cruzada no treino):")
print(df_resultados[['Modelo', 'Acurácia', 'Precisão', 'Recall', 'F1-Score']].round(3))

# Exibir hiperparâmetros
print("\n🔍 Melhores Hiperparâmetros:")
for i, row in df_resultados.iterrows():
    print(f"{row['Modelo']}: {row['Melhores Hiperparâmetros']}")



🔧 Treinando: Random Forest


c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConvers


🔧 Treinando: SVM


c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataC


🔧 Treinando: KNN


...


🔧 Treinando: Decision Tree

🔧 Treinando: Logistic Regression


c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataC


🔧 Treinando: Gradient Boosting


c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\cesar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.p


📊 Resultados dos Modelos (validação cruzada no treino):
                Modelo  Acurácia  Precisão  Recall  F1-Score
5  Logistic Regression     0.805     0.797   0.805     0.799
3              XGBoost     0.802     0.793   0.802     0.796
1                  SVM     0.801     0.791   0.801     0.793
6    Gradient Boosting     0.798     0.788   0.798     0.791
0        Random Forest     0.797     0.786   0.797     0.788
4        Decision Tree     0.786     0.780   0.786     0.783
2                  KNN     0.768     0.763   0.768     0.766

🔍 Melhores Hiperparâmetros:
Logistic Regression: {'C': 1, 'penalty': 'l2'}
XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
SVM: {'C': 1, 'kernel': 'rbf'}
Gradient Boosting: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Random Forest: {'max_depth': 10, 'n_estimators': 100}
Decision Tree: {'criterion': 'entropy', 'max_depth': 5}
KNN: {'metric': 'euclidean', 'n_neighbors': 7}
